In [ ]:
import pandas as pd

!pip install chart_studio

!pip install ipyleaflet

location_dataframe = pd.read_csv("https://raw.githubusercontent.com/mwigham/linked_data_notebooks/main/locations.csv")

## Plot using Plotly

In [ ]:
import chart_studio
import plotly
from chart_studio import plotly as py
import plotly.graph_objects as go

coords = pd.concat([location_dataframe['Location'],
                    location_dataframe['Latitude'], 
                    location_dataframe['Longitude'], 
                    location_dataframe['Counts']], axis=1)
locations = []
colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']


locations.append(go.Scattergeo(
    lon = coords['Longitude'],
    lat = coords['Latitude'],
    text = [f"{coords['Location'][i]} {coords['Counts'][i]}" for i in range(len(coords['Location']))],
    marker = dict(
        size = coords['Counts']/60,
        opacity = .4,
        line = dict(width = 0)
    ),
) )

locations[0]['mode'] = 'markers'

                
layout = go.Layout(
    title = 'Standardised concert recording locations in the MOZ collection',
    geo = dict(
        resolution = 110,
        scope = 'europe',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'mercator'
        ),
        lonaxis = dict( range= [ 3, 8] ),
        lataxis = dict( range= [ 50,54 ] ),
        
    ),
    legend = dict(
           traceorder = 'reversed'
    ),
    width = 1000,
    height = 800,
)


fig = go.Figure(layout=layout, data=locations)

plotly.offline.iplot(fig, validate=False, filename='maritest')

## Plot using Flourish
Export data and copy into a Flourish map
Example map is available at https://public.flourish.studio/visualisation/14539726/

In [ ]:
location_dataframe.to_csv("locations.csv")


## Plot using ipyleaflet

In [ ]:
from ipywidgets import HTML
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(53, 4),
    zoom=6, 
)


for index, row in location_dataframe.iterrows():
    circle_marker = CircleMarker()
    circle_marker.location = (row['Latitude'], row['Longitude'])
    circle_marker.radius = int(row['Counts']/100)
    circle_marker.color = "red"
    circle_marker.fill_color = "red"

    m.add_layer(circle_marker)
    
    # Popup with a given location on the map:
    message = HTML()
    message.value = f"{row['Counts']} concerts"
    message.description = str(row['Location'])

    # Popup associated to a layer
    circle_marker.popup = message

m